In [ ]:
import pyPamtra
import numpy as np
import matplotlib.pyplot as plt
import sys
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import scipy
import random
import pandas as pd
from scipy.interpolate import UnivariateSpline
import glob
import time
import copy
import os
from tqdm import tqdm


In [2]:
params_from_dataset_list = pd.read_feather('./params_from_dataset_10spec.feather') # single mode data 
new_df = pd.DataFrame({'Ze_X': [], 'Ze_Ka':[], 'Ze_W':[], 
                                       'mdv_X':[], 'mdv_Ka':[], 'mdv_W':[],
                                       'sw_X':[], 'sw_Ka':[], 'sw_W':[],
                                       'sk_X':[], 'sk_Ka':[], 'sk_W':[],
                                       'spectra_X': [], 'spectra_Ka':[], 'spectra_W':[]})
edr = 1e-4

params_to_save = pd.DataFrame({
    'M0':[], 'M0_2':[], 
    'Deff':[], 'Deff_2':[], 
    'mu':[], 'mu_2':[], 
    'a_mass_size':[], 'a_mass_size_2':[],
    'b_mass_size':[], 'b_mass_size_2':[],
    'alpha_area_size':[], 'alpha_area_size_2':[],
    'beta_area_size':[], 'beta_area_size_2':[],

    'temperature':[], 'wind_w_W':[], 'fft_len_W':[], 'v_nyq_W':[], 'beamwidth_deg_W':[], 'integration_time_W':[], 'freq_W':[],
    'noise_level_Ka':[], 'fft_len_Ka':[], 'v_nyq_Ka':[], 'beamwidth_deg_Ka':[], 'integration_time_Ka':[], 'freq_Ka':[],
    'fft_len_X':[], 'v_nyq_X':[], 'beamwidth_deg_X':[], 'integration_time_X':[], 'freq_X':[]
})

# combinations of single-mode spectra
i = [0, 0, 2, 1, 6, 3, 0, 2, 2, 6, 6, 1, 1]
j = [4, 5, 8, 7, 1, 1, 1, 1, 7, 8, 9, 4, 8]


In [3]:
rho_air = 1.2

for a in range(len(i)): 
    params_from_dataset_m1 = params_from_dataset_list.iloc[i[a]]
    params_from_dataset_m2 = params_from_dataset_list.iloc[j[a]]

    ########### DEFINE MICROPHYSICAL PARAMETERS ##################
    aspect_ratio = 0.6
    canting_angle = 0.

    ######### RESHAPE PARAMETERS CORRECTLY ##################      

    pam = pyPamtra.pyPamtra()
    pam.df.addHydrometeor(('ice0', #name
                            aspect_ratio, #aspect ratio, <1 means oblate
                            -1, #phase: -1=ice, 1=liq
                            -99.,#200, #density
                            params_from_dataset_m1['a_mass_size'], #a parameter of mass-size
                            params_from_dataset_m1['b_mass_size'], #b parameter of mass-size
                            params_from_dataset_m1['alpha_area_size'], #alpha parameter of cross-section area - size relation
                            params_from_dataset_m1['beta_area_size'], #beta parameter of cross-section area - size relation
                            12, # moment provided in input file
                            200, #number of discrete size bins (internally, nbins+1 is used)
                            'mgamma', #name of psd
                            -99., #1st parameter of psd
                            -99.,#2nd parameter of psd
                            params_from_dataset_m1['mu'], #3rd parameter of psd
                            1., #4th parameter of psd
                            1e-5, #min diameter
                            1e-2, # max diameter
                            'ss-rayleigh-gans',
                            'heymsfield10_particles', 
                            canting_angle)) #canting angle of hydrometeors, only for Tmatrix and SSRG

    pam.df.addHydrometeor(('ice1', #name
                            aspect_ratio, #aspect ratio, <1 means oblate
                            -1, #phase: -1=ice, 1=liq
                            -99.,#200, #density
                            params_from_dataset_m2['a_mass_size'], #a parameter of mass-size
                            params_from_dataset_m2['b_mass_size'], #b parameter of mass-size
                            params_from_dataset_m2['alpha_area_size'], #alpha parameter of cross-section area - size relation
                            params_from_dataset_m2['beta_area_size'], #beta parameter of cross-section area - size relation
                            12, # moment provided in input file
                            200, #number of discrete size bins (internally, nbins+1 is used)
                            'mgamma', #name of psd
                            -99., #1st parameter of psd
                            -99.,#2nd parameter of psd
                            params_from_dataset_m2['mu'], #3rd parameter of psd
                            1., #4th parameter of psd
                            1e-5, #min diameter
                            1e-2, # max diameter
                            'ss-rayleigh-gans',
                            'heymsfield10_particles', 
                            0)) #canting angle of hydrometeors, only for Tmatrix and SSRG

    pam = pyPamtra.importer.createUsStandardProfile(pam,hgt_lev=np.array([np.linspace(999,1001,2).tolist()]))
    pam.p['temp_lev'][:]=params_from_dataset_m1['temperature']+273.15
    pam.p['relhum_lev'][:]=90.
    pam.set["verbose"] = -1

    pam.p["hydro_n"][0,0,:,0] = params_from_dataset_m1['M0']/rho_air
    pam.p["hydro_n"][0,0,:,1] = params_from_dataset_m2['M0']/rho_air
    pam.p["hydro_reff"][0,0,:,0] = 1/2*params_from_dataset_m1['Deff']
    pam.p["hydro_reff"][0,0,:,1] = 1/2*params_from_dataset_m2['Deff']
        

    pam.p["wind_w"][:] = params_from_dataset_m1['wind_w_W']
    pam.nmlSet["radar_mode"] = "spectrum"
    pam.nmlSet['save_psd']=True
    pam.nmlSet["radar_noise_distance_factor"] = 1.0
    pam.nmlSet["radar_save_noise_corrected_spectra"]=  False
    pam.nmlSet['passive'] = False
    pam.nmlSet['radar_airmotion'] = False
    pam.nmlSet['radar_aliasing_nyquist_interv'] = 1
    pam.nmlSet['obs_height'] = 0
    pam.nmlSet['hydro_adaptive_grid'] = False
    pam.nmlSet['radar_allow_negative_dD_dU'] = True

    pam.nmlSet['radar_nfft']= params_from_dataset_m1['fft_len_W']
    pam.nmlSet['radar_max_v']= params_from_dataset_m1['v_nyq_W']
    pam.nmlSet['radar_min_v']= -params_from_dataset_m1['v_nyq_W']

    vel0 = np.linspace(-params_from_dataset_m1['v_nyq_W'],params_from_dataset_m1['v_nyq_W'],params_from_dataset_m1['fft_len_W'],endpoint=False)
    dv0 = vel0[1]-vel0[0]
    pam.nmlSet['radar_pnoise0'] = 10*np.log10(np.array(params_from_dataset_m1['noise_level_Ka']))+10*np.log10(dv0*params_from_dataset_m1['fft_len_Ka'])

    pred = {}

    for freq in ['X','Ka','W']:
        pam.addSpectralBroadening(edr, 10, params_from_dataset_m1['beamwidth_deg_%s'%freq], params_from_dataset_m1['integration_time_%s'%freq], params_from_dataset_m1['freq_%s'%freq], kolmogorov=0.5)
        pam.runPamtra(params_from_dataset_m1['freq_%s'%freq])
        pred['spectra_%s'%freq]=[pam.r['radar_spectra'][0,0,0,0,0,:]-10*np.log10(dv0)]
        pred['moments_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,:].tolist() 
        pred['Ze_%s'%freq] = pam.r["Ze"][0,0,0,0,0,0]
        pred['mdv_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,0]
        pred['sw_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,1]
        pred['sk_%s'%freq] = pam.r["radar_moments"][0,0,0,0,0,0,2]


    new_df.loc[a]=pd.Series({
    'spectra_X': pred['spectra_X'][0], 'spectra_Ka':pred['spectra_Ka'][0], 'spectra_W':pred['spectra_W'][0]
                                                })
    
    params_to_save.loc[a] = pd.Series({
        'M0': params_from_dataset_m1['M0'],
        'M0_2': params_from_dataset_m2['M0'],
        'Deff': params_from_dataset_m1['Deff'],
        'Deff_2': params_from_dataset_m2['Deff'],
        'mu': params_from_dataset_m1['mu'],
        'mu_2': params_from_dataset_m2['mu'],
        'a_mass_size': params_from_dataset_m1['a_mass_size'],
        'a_mass_size_2': params_from_dataset_m2['a_mass_size'],
        'b_mass_size': params_from_dataset_m1['b_mass_size'],
        'b_mass_size_2': params_from_dataset_m2['b_mass_size'],
        'alpha_area_size': params_from_dataset_m1['alpha_area_size'],
        'alpha_area_size_2': params_from_dataset_m2['alpha_area_size'],
        'beta_area_size': params_from_dataset_m1['beta_area_size'],
        'beta_area_size_2': params_from_dataset_m2['beta_area_size'],

        'temperature': params_from_dataset_m1['temperature'],
        'wind_w_W': params_from_dataset_m1['wind_w_W'],
        'fft_len_W': params_from_dataset_m1['fft_len_W'],
        'v_nyq_W': params_from_dataset_m1['v_nyq_W'],
        'beamwidth_deg_W': params_from_dataset_m1['beamwidth_deg_W'],
        'integration_time_W': params_from_dataset_m1['integration_time_W'],
        'freq_W': params_from_dataset_m1['freq_W'],

        'noise_level_Ka': params_from_dataset_m1['noise_level_Ka'],
        'fft_len_Ka': params_from_dataset_m1['fft_len_Ka'],
        'v_nyq_Ka': params_from_dataset_m1['v_nyq_Ka'],
        'beamwidth_deg_Ka': params_from_dataset_m1['beamwidth_deg_Ka'],
        'integration_time_Ka': params_from_dataset_m1['integration_time_Ka'],
        'freq_Ka': params_from_dataset_m1['freq_Ka'],

        'fft_len_X': params_from_dataset_m1['fft_len_X'],
        'v_nyq_X': params_from_dataset_m1['v_nyq_X'],
        'beamwidth_deg_X': params_from_dataset_m1['beamwidth_deg_X'],
        'integration_time_X': params_from_dataset_m1['integration_time_X'],
        'freq_X': params_from_dataset_m1['freq_X'],

    })

In [14]:
new_df.to_feather(target_save_path)

In [7]:
params_to_save.to_feather(params_save_path)